In [ ]:
#Import dependencies 
import pandas as pd
import numpy as np
pd.set_option('max_colwidth', 400)

Extract the Data

In [ ]:
#Read the Shark Tank data into a dataframe
sharktank_df = pd.read_csv('shark_tank_raw_data/Shark Tank US dataset.csv')
sharktank_df.head()


Perform initial data cleaning

In [ ]:
sharktank_df.info()

In [ ]:
sharktank_df.columns

In [ ]:
#remove unneccesary columns
sharktank_df_clean = sharktank_df[['Season Number', 'Startup Name', 'Episode Number', 'Pitch Number',
       'Original Air Date', 'Industry',
       'Business Description', 'Company Website', 'Pitchers Gender',
       'Pitchers City', 'Pitchers State', 'Entrepreneur Names', 'Multiple Entrepreneurs',
       'Original Ask Amount', 'Original Offered Equity', 'Valuation Requested',
       'Got Deal', 'Total Deal Amount', 'Total Deal Equity', 'Deal Valuation',
       'Number of Sharks in Deal', 'Investment Amount Per Shark',
       'Equity Per Shark', 'Barbara Corcoran Investment Amount',
       'Barbara Corcoran Investment Equity', 'Mark Cuban Investment Amount',
       'Mark Cuban Investment Equity', 'Lori Greiner Investment Amount',
       'Lori Greiner Investment Equity', 'Robert Herjavec Investment Amount',
       'Robert Herjavec Investment Equity', 'Daymond John Investment Amount',
       'Daymond John Investment Equity', 'Kevin O Leary Investment Amount',
       'Kevin O Leary Investment Equity', 'Guest Investment Amount',
       'Guest Investment Equity', 'Guest Name', 'Barbara Corcoran Present',
       'Mark Cuban Present', 'Lori Greiner Present', 'Robert Herjavec Present',
       'Daymond John Present', 'Kevin O Leary Present', 'Guest Present']]

sharktank_df_clean.head()

In [ ]:
#transform dates to dt format
sharktank_df_clean['Original Air Date'] = pd.to_datetime(sharktank_df_clean['Original Air Date'], format='%d-%b-%y')


In [ ]:
#check that dates converted correctly
sharktank_df_clean.head()

In [ ]:
#transform 1 and 0 values to boolean
sharktank_df_clean[['Got Deal', 
                    'Barbara Corcoran Present', 
                    'Mark Cuban Present', 
                    'Lori Greiner Present', 
                    'Robert Herjavec Present', 
                    'Daymond John Present',
                    'Kevin O Leary Present', 
                    'Guest Present']] = sharktank_df_clean[['Got Deal',
                                                            'Barbara Corcoran Present', 
                                                            'Mark Cuban Present', 
                                                            'Lori Greiner Present', 
                                                            'Robert Herjavec Present', 
                                                            'Daymond John Present', 
                                                            'Kevin O Leary Present', 
                                                            'Guest Present']].astype('bool')

In [ ]:
#check that values converted to boolean correctly
sharktank_df_clean.head()

In [ ]:
# Drop any duplicate values from pitch number (stack overflow https://stackoverflow.com/questions/23667369/drop-all-duplicate-rows-across-multiple-columns-in-python-pandas)
sharktank_df_clean.drop_duplicates(subset=['Pitch Number'], keep=False, inplace=True)

# Replace any NaN values with "none" (Xpert learning assistant)
#sharktank_df_clean.fillna("none", inplace=True)

sharktank_df_clean.info()

Pitchers Demographics Table Cleaning (Anna)

In [ ]:
#Create the pitcher demographics table from the cleaned sharktank df
#Reorder the column names
pitcher_demo_df = sharktank_df_clean[["Pitch Number",
                                    "Multiple Entrepreneurs", 
                                    "Entrepreneur Names",
                                    "Pitchers Gender",
                                    "Pitchers City",
                                    "Pitchers State",
                                    "Industry"]]

pitcher_demo_df.head()

In [ ]:
#change the multiple entrepreneurs column to boolean
pitcher_demo_df["Multiple Entrepreneurs"] = pitcher_demo_df["Multiple Entrepreneurs"].astype('bool')

pitcher_demo_df.head()

In [ ]:
# replace any null values in the Entrepreneur Names, Gender, City, State Columns to "unknown"
# sources: https://stackoverflow.com/questions/34913590/fillna-in-multiple-columns-in-place-in-python-pandas

replace_na_strings = pitcher_demo_df.select_dtypes(object).columns
pitcher_demo_df[replace_na_strings] = pitcher_demo_df[replace_na_strings].fillna("Unknown")


In [ ]:
# Create a "Entrepreneur 1 Name" and "Entrepreneur 2 Name" column with the first and last names from the "name" column. 
# Only split columns which have a comma, or an "and" to signify multiple pitchers
# Code was generated with assistance from Bootcamp Spot Xpert Learning Assistant

for index, row in pitcher_demo_df.iterrows():
    if "," in row["Entrepreneur Names"]:
         names = row['Entrepreneur Names'].split(',')
         pitcher_demo_df.at[index, 'Entrepreneur_1_Name'] = names[0]
         pitcher_demo_df.at[index, 'Entrepreneur_2_Name'] = names[1] if len(names) > 1 else None

    elif "and" in row["Entrepreneur Names"]:
         names = row['Entrepreneur Names'].split('and ')
         pitcher_demo_df.at[index, 'Entrepreneur_1_Name'] = names[0]
         pitcher_demo_df.at[index, 'Entrepreneur_2_Name'] = names[1] if len(names) > 1 else None

    else: pitcher_demo_df.at[index, 'Entrepreneur_1_Name'] = pitcher_demo_df.at[index, 'Entrepreneur Names']


pitcher_demo_df.head()

In [ ]:
#fix any instances where multiple enterpreneurs column is incorrectly classified
#Source: https://www.programiz.com/python-programming/pandas/handle-wrong-data

for row in pitcher_demo_df.index:
    multiple_check = pitcher_demo_df.loc[row, "Entrepreneur_2_Name"]
    if pd.isnull(multiple_check):
        pitcher_demo_df.loc[row, "Multiple Entrepreneurs"] = False
    else: 
        pitcher_demo_df.loc[row, "Multiple Entrepreneurs"] = True

pitcher_demo_df.fillna("none", inplace=True)

pitcher_demo_df.head()


In [ ]:
#reorder columns
pitcher_demo_df_cleaned = pitcher_demo_df[["Pitch Number", 
                                   "Multiple Entrepreneurs", 
                                   "Entrepreneur_1_Name",
                                   "Entrepreneur_2_Name",
                                    "Pitchers Gender",
                                    "Pitchers City",
                                    "Pitchers State",
                                    "Industry"]]

#rename columns to simplify
pitcher_demo_df_cleaned.rename(columns= {"Pitch Number":"Pitch_Number",
                                         "Multiple Entrepreneurs": "Multiple_Entrepreneurs",
                                         "Pitchers Gender":"Gender",
                                         "Pitchers City": "City",
                                         "Pitchers State": "State"}, inplace =True)

pitcher_demo_df_cleaned.fillna("none", inplace=True)

pitcher_demo_df_cleaned.head()

In [ ]:
#Check datatypes and null counts one last time before exporting to CSV

pitcher_demo_df_cleaned.info()

In [ ]:
pitcher_demo_df_cleaned.to_csv("pitch_demo.csv", encoding = 'utf8', index=False)

Pitchers Demographics Table (Data cleaning by Anna Bitzer)

A subset of columns from the sharktank_df_clean dataset were used to create a Pitchers Demographics table. Several data cleaning steps were taken to transform and prepare the data for storage in a SQL database.
 -  The "Multiple Entrepreneurs" column datatype was switched to boolean.
 -  Null values in all string columns (name, gender, city, state, industry) were replaced with "Unknown.
 -  The "Entrepreneur Names" column, which could contain multiple entrepreneurs, was split into new columns "Entrepreneur 1 Name" and "Entrepreneur 2 Name", splitting at a comma or the word "and". 
 -  Some instances of the "Multiple Entrepreneurs" column were found to be incorrect. They were corrected using a conditional that checked if a second entrepreneur was present in the "Entrepreneur 2 Name" column.
 -  Finally, the columns were reorderd and some were renamed.

The final dataframe was exported to csv, for upload into a SQL database with the other tables.

Sources Used: 
https://stackoverflow.com/questions/34913590/fillna-in-multiple-columns-in-place-in-python-pandas, https://www.programiz.com/python-programming/pandas/handle-wrong-data, and BootcampSpot Xpert Learning Assistant for formatting itterows code to split names.



Shark Demogrpahics Table Cleaning

In [ ]:
# Sharks Demographic
Shark_Name = sharktank_df_clean["Guest Name"].dropna().unique()
sharks_demographic_df = pd.DataFrame({'Shark_Name':Shark_Name})
# sharks_demographic_df.index.name="Shark_ID"
sharks_demographic_df.reset_index(inplace=True)
sharks_demographic_df = sharks_demographic_df.rename(columns={'index': 'Shark_ID'})

# Data discrepancy: 26 Nirav Tolia; 27	Nirv Tolia
sharks_demographic_df.loc[27, 'Shark_Name'] = "Nirav Tolia"
sharks_demographic_df["Shark_Name"].drop_duplicates()
# Add index for shark demo table
sharks_demographic_df['Shark_ID'] = range(1, len(sharks_demographic_df) + 1)

sharks_demographic_df


In [ ]:
# Adding externally sourced gender information to shark demo table:
gender =  ['Male','Male','Male','Male','Male','Male','Male','Male','Male','Male','Male','Female','Female','Female','Male','Male','Male','Male','Female','Female','Female','Male','Female','Male','Female','Male','Female','Female','Male','Female','Male','Female','Male','Male']
sharks_demographic_df['Gender'] = gender
sharks_demographic_df

Pitch Info Table Cleaning (Tianyue)

In [ ]:
# Creating a dataframe for Deal Info:
deal_df = sharktank_df_clean[[ 'Pitch Number','Got Deal','Total Deal Amount','Total Deal Equity','Original Offered Equity','Valuation Requested', 'Deal Valuation','Investment Amount Per Shark', 'Equity Per Shark','Number of Sharks in Deal', 'Barbara Corcoran Investment Amount',
                    'Barbara Corcoran Investment Equity', 'Mark Cuban Investment Amount','Mark Cuban Investment Equity', 'Lori Greiner Investment Amount','Lori Greiner Investment Equity', 
                    'Robert Herjavec Investment Amount', 'Robert Herjavec Investment Equity','Daymond John Investment Amount','Daymond John Investment Equity', 'Kevin O Leary Investment Amount',
                    'Kevin O Leary Investment Equity', 'Guest Investment Amount', 'Guest Investment Equity']]
 

deal_df.columns = deal_df.columns.str.replace(' ', '_')
deal_df['Deal_ID'] = range(1, len(deal_df) + 1)
deal_df.head(20)


In [ ]:
# Demonstrating database characteristics: Shark investment amount, equity columns contain a significant amount of NaN values that required transposing
deal_df.count()
deal_df.dropna().count()

In [ ]:
numeric_columns = deal_df.select_dtypes(include=['float64', 'int64'])
deal_df[numeric_columns.columns] = numeric_columns.fillna(0)
deal_df.head()


In [ ]:
#Creating a dataframe for Pitch Info: 
pitch_df= sharktank_df_clean[['Pitch Number', 'Episode Number', 'Season Number','Business Description', 'Got Deal']]
pitch_df.columns = pitch_df.columns.str.replace(' ', '_')

pitch_df.fillna("none", inplace=True)
pitch_df.head()

In [ ]:
# Data use case 1: Filter for successful deals and merge with "Pitch_df" to create visualizations
# Ex. Use Pitch_Number, Total_Deal_Amount, Total_Deal_Equity, Industry
deal_y_df = deal_df[deal_df['Got_Deal']==True]
deal_y_df


pitch_deal_merge_df = pd.merge(deal_y_df, pitcher_demo_df_cleaned, on="Pitch_Number", how="outer")
pitch_deal_merge_df

In [ ]:
# Data use case 2: Merge pitcher table and deal table to analyze industry trend
import matplotlib.pyplot as plt

industry_df = pitch_deal_merge_df[['Industry', 'Pitch_Number']]
industry_df
industry_count_df = industry_df.groupby('Industry').count()
industry_count_df
industry_count_df = industry_df['Industry'].value_counts()
industry_count_df.plot(kind='pie', figsize=(8, 8), autopct='%1.1f%%', startangle=160)

plt.xlabel("Industry")
plt.ylabel("Count of Pitches")
plt.title("Invested Pitches by Industry")
plt.show()

In [ ]:
# Data use Case 3: Merge pitcher table and deal table to analyze gender factors
import matplotlib.pyplot as plt

industry_df = pitch_deal_merge_df[['Gender', 'Pitch_Number']]
industry_df
industry_count_df = industry_df.groupby('Gender').count()
industry_count_df
# industry_df.plot(kind='pie', figsize=(8,8), autopct='%1.1f%%')
industry_count_df = industry_df['Gender'].value_counts()
industry_count_df.plot(kind='pie', figsize=(8, 8), autopct='%1.1f%%', startangle=160)

plt.xlabel("Industry")
plt.ylabel("Count of Gender")
plt.title("Invested Pitches by Gender")
plt.show()

In [ ]:
# Data use case 4: Demonstrate each shark's top 10 investments and pitch numbers
pitch_deal_merge_df.Barbara_Corcoran_Investment_Equity.dropna().sort_values(ascending=False)[:10].plot.bar(figsize=[16,4],rot=90)
plt.xlabel("Pitch Number")
plt.ylabel("Equity %")
plt.title("Barbara's Top 10 Investment in Equity")
plt.show()

pitch_deal_merge_df.Mark_Cuban_Investment_Equity.dropna().sort_values(ascending=False)[:10].plot.bar(figsize=[16,4],rot=90)
plt.xlabel("Pitch Number")
plt.ylabel("Equity %")
plt.title("Mark Cuban's Top 10 Equity Investment")
plt.show()

In [ ]:
# Data use Case #5: Shark table to evaluate sharks' gender impact on pitch success
gender_counts = sharks_demographic_df['Gender'].value_counts()
gender_counts
# # Create a pie chart based on the count of genders

plt.figure(figsize=(8, 8))
gender_counts.plot(kind='pie', figsize=(8, 8), autopct='%1.1f%%', startangle=160)
plt.xlabel("Industry")
plt.ylabel("Count of Gender")
plt.title("Invested Pitches by Gender")
plt.show()

# Demo purpose only: print out sharks' gender counts: Male vs Female
sharks_demographic_df[sharks_demographic_df['Gender'] == 'Male'].count()
sharks_demographic_df[sharks_demographic_df['Gender'] == 'Female'].count()


In [ ]:
# CSV Exporting code (Only run when exporting)
deal_df.to_csv("deal_df.csv", index=False)

In [ ]:
# CSV Exporting code (Only run when exporting)
pitch_df.to_csv("pitch_df.csv", index=False)
